In [ ]:
import pickle
from graph import Graph, Part
from typing import List, Set, Tuple, Dict
from sklearn.model_selection import train_test_split

with open("data/graphs.dat", "rb") as file:
    all_graphs: List[Graph] = pickle.load(file)
    X_train, X_temp, y_train, y_temp = train_test_split(
        list(map(lambda g: g.get_parts(), all_graphs)),
        all_graphs,
        test_size=0.3,
        random_state=0,
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=0
    )

In [ ]:
from node import Node
from collections import defaultdict
import networkx as nx


# Create frozen set of part ids
def edge(part1: Part, part2: Part) -> Set[int]:
    return frozenset({part1.get_part_id(), part2.get_part_id()})


# Count how often parts appear together
def extract_pairs(graphs: List[Graph]) -> Dict[Set[Part], int]:
    pairs = defaultdict(int)
    for graph in graphs:
        for part1 in graph.get_parts():
            for part2 in graph.get_parts():
                pairs[edge(part1, part2)] += 1
    return pairs


# Count degrees of parts and existing edges
def extract_edges(graphs: List[Graph]) -> Tuple[Dict[Part, int], Dict[Set[Part], int]]:
    degrees = defaultdict(list)
    edges = defaultdict(int)
    for graph in graphs:
        for node, neighbors in graph.get_edges().items():
            degrees[node.get_part().get_part_id()].append(len(neighbors))
            for neighbor in neighbors:
                edges[edge(node.get_part(), neighbor.get_part())] += 1
    return degrees, edges


# Edge priority before training is the ratio of edges to occurrences for a pair of parts
def init_edge_priority(
    pair_count: Dict[Set[Part], int], edge_count: Dict[Set[Part], int]
):
    edge_priority = defaultdict(float)
    for pair, count in pair_count.items():
        edge_priority[pair] = edge_count[pair] / count
    return edge_priority


# Check whether the graph described by edges is a subgraph of the target graph
def edges_are_subgraph(edges: List[Tuple[Part, Part]], target: Graph) -> bool:
    g = Graph()
    for edge0, edge1 in edges:
        g.add_undirected_edge(edge0, edge1)
    return nx.algorithms.isomorphism.GraphMatcher(
        target.to_nx(), g.to_nx(), node_match=lambda a, b: a == b
    ).subgraph_is_isomorphic()


# Find the first edge that is not part of the target graph
def first_wrong_edge(
    edges: List[Tuple[Part, Part]], target: Graph
) -> Tuple[Part, Part]:
    # Use binary search and edges_are_subgraphs to find the first wrong edge:
    min = 1
    max = len(edges)
    while min < max:
        mid = (min + max) // 2
        if edges_are_subgraph(edges[:mid], target):
            min = mid + 1
        else:
            max = mid
    return min - 1


class InstanceBased:
    def __init__(
        self,
        y: List[Graph],
        order_train_rate=0.1,
        order_train_epochs=2,
        edge_train_rate=0.001,
        edge_train_epochs=10,
        edge_pickle_load=False,
        edge_pickle_store=False,
    ):
        pairs_count = extract_pairs(y)
        degrees, edge_count = extract_edges(y)
        self.edge_priority = init_edge_priority(pairs_count, edge_count)
        self.avgDegrees = {part: sum(deg) / len(deg) for (part, deg) in degrees.items()}
        self.maxDegrees = {part: max(deg) for (part, deg) in degrees.items()}
        self.sort_priority = self.avgDegrees.copy()
        self.train_order(y, order_train_rate, order_train_epochs)
        if edge_pickle_load:
            path = (
                edge_pickle_load
                if isinstance(edge_pickle_load, str)
                else "data/edge_priority.dat"
            )
            with open(path, "rb") as file:
                self.edge_priority = pickle.load(file)
        else:
            self.train_edges(y, edge_train_rate, edge_train_epochs)
        if edge_pickle_store:
            path = (
                edge_pickle_store
                if isinstance(edge_pickle_store, str)
                else "data/edge_priority.dat"
            )
            with open(path, "wb") as file:
                pickle.dump(self.edge_priority, file)

    # Sort parts to allow building tree sequentially
    def predict_order(self, x: Set[Part]) -> List[Part]:
        return sorted(x, key=lambda n: self.sort_priority[n.get_part_id()], reverse=1)

    # Train sorting to enable "dumbbell" graphs
    def train_order(self, graphs: List[Graph], train_rate, epochs):
        print(f"Possible accuracy when sequentially building a tree using sort method")
        for epoch in range(epochs):
            print(
                f"Train order, Epoch {epoch}: {100 * self.evaluate_order(graphs, train_rate) / len(graphs)}% accuracy"
            )
        print(
            f"Train order, Epoch {epochs}: {100 * self.evaluate_order(graphs) / len(graphs)}% accuracy"
        )

    # (Train and) evaluate supprt of graphs
    def evaluate_order(self, graphs: List[Graph], train_rate=False) -> int:
        compatible_graphs = 0
        for graph in graphs:
            compatible = True
            edges = {
                node: edges.copy() for (node, edges) in graph.get_edges().items()
            }  # Copy edges
            nodes: List[Node] = sorted(
                graph.get_nodes(),
                key=lambda n: self.sort_priority[n.get_part().get_part_id()],
            )  # Sort nodes reversed

            # Check whether leaves can be removed one by one using reversed sorting order
            for node in nodes[:-1]:
                if len(edges[node]) != 1:
                    # If not a leaf, it should've appeared later
                    compatible = False
                    if train_rate:
                        # If training, adapt priority accordingly
                        self.sort_priority[node.get_part().get_part_id()] += train_rate
                    break
                # Remove the leaf
                edges[edges[node][0]].remove(node)
            compatible_graphs += compatible
        return compatible_graphs

    def _next_correct_edge(
        self, edges: List[Tuple[Part, Part]], part: Part, target: Graph
    ) -> Tuple[Part, Part]:
        parts = [edges[0][0]] + [edge[1] for edge in edges]
        candidates = sorted(
            [(p, part) for p in parts],
            key=lambda p: self.edge_priority[edge(p[0], p[1])],
            reverse=True,
        )
        for candidate in candidates:
            if edges_are_subgraph(edges + [candidate], target):
                return candidate

    # Sequentially builds graph after sorting parts
    def createGraph(
        self,
        unordered_parts: Set[Part],
        avgDegreeInfluence=0.05,
        maxDegreeInfluence=0.5,
        target=None,
        train_rate=None,
    ) -> Graph:
        parts = self.predict_order(unordered_parts)
        edges = [(parts[0], parts[1])]
        g = Graph()
        g.add_undirected_edge(parts[0], parts[1])
        for i in range(2, len(parts)):
            best_edge = max(
                [(p, parts[i]) for p in parts[:i]],
                default=(parts[0], parts[i]),
                key=lambda p: self.edge_priority[edge(p[0], p[1])]
                + avgDegreeInfluence  # Prefer nodes that require more neighbors
                * (self.avgDegrees[p[0].get_part_id()] - g.get_degree(p[0]))
                - maxDegreeInfluence  # Dont exceed maxDegree
                * (g.get_degree(p[0]) == self.maxDegrees[p[0].get_part_id()]),
            )
            edges.append(best_edge)
            g.add_undirected_edge(best_edge[0], best_edge[1])
        if target and train_rate and g != target:
            first_wrong = first_wrong_edge(edges, target)
            correct = self._next_correct_edge(
                edges[:first_wrong], parts[first_wrong + 1], target
            )
            self.edge_priority[edge(correct[0], correct[1])] += train_rate
            # print(edges)
            # g.draw()
            # print(f"{correct} instead of {edges[first_wrong]}")
            # target.draw()
            return None
        return g

    def train_edges(self, graphs: List[Graph], train_rate, epochs):
        print(f"Training edge priorities {epochs} epochs: ", end="")
        for epoch in range(epochs):
            for graph in graphs:
                self.createGraph(graph.get_parts(), target=graph, train_rate=train_rate)
            print(f"{epoch + 1} ", end="")
        print("done")

In [ ]:
ib = InstanceBased(y_train, edge_pickle_store=True)
print(f"Order Validation: {ib.evaluate_order(y_val) / len(y_val)}% accuracy\n")

y_train_prev_right = []
y_train_prev_wrong = []
y_val_prev_right = []
y_val_prev_wrong = []

correct_train = 0
for i in range(len(y_train)):
    if y_train[i] == ib.createGraph(X_train[i]):
        correct_train += 1
        y_train_prev_right.append(y_train[i])
    else:
        y_train_prev_wrong.append(y_train[i])
print(f"train_err: {correct_train / len(y_train)}")
correct_val = 0
for i in range(len(y_val)):
    if y_val[i] == ib.createGraph(X_val[i]):
        correct_val += 1
        y_val_prev_right.append(y_val[i])
    else:
        y_val_prev_wrong.append(y_val[i])
print(f"val_err: {correct_val / len(y_val)}")

In [ ]:
ib = InstanceBased(y_train, edge_pickle_load=True)

correct_train = 0
for i in range(len(y_train)):
    if y_train[i] == ib.createGraph(X_train[i]):
        correct_train += 1
print(f"train_err: {correct_train / len(y_train)}")
correct_val = 0
for i in range(len(y_val)):
    if y_val[i] == ib.createGraph(X_val[i]):
        correct_val += 1
print(f"val_err: {correct_val / len(y_val)}")

### Current problems:
train[5]: 114 is added twice to 36 (fewer but higher probability) instead of once to 36 and once to 182

train[14]: Chosen edge is way more probable, would require much more context information

train[35]: 218 on 301 instead of duplicate at 1156

train_err: 0.9244654973754961
val_err: 0.896057347670250


Especially slow: train[6283], train[4288], 


Decomposing the error rate:
0.6% incorrect orderings
1.9% incorrect saturation (maxDegree)


In [ ]:
from itertools import permutations
import numpy as np

def evaluate( data_set: List[Tuple[Set[Part], Graph]]) -> float:
    """
    Evaluates a given prediction model on a given data set.
    :param model: prediction model
    :param data_set: data set
    :return: evaluation score (for now, edge accuracy in percent)
    """
    sum_correct_edges = 0
    edges_counter = 0

    for input_parts, target_graph in data_set:
        predicted_graph = ib.createGraph(input_parts)

        edges_counter += len(input_parts) * len(input_parts)
        sum_correct_edges += edge_accuracy(predicted_graph, target_graph)

    return sum_correct_edges / edges_counter * 100


def edge_accuracy(predicted_graph: Graph, target_graph: Graph) -> int:
    """
    Returns the number of correct predicted edges.
    :param predicted_graph:
    :param target_graph:
    :return:
    """
    assert len(predicted_graph.get_nodes()) == len(target_graph.get_nodes()), 'Mismatch in number of nodes.'
    assert predicted_graph.get_parts() == target_graph.get_parts(), 'Mismatch in expected and given parts.'

    best_score = 0

    # Determine all permutations for the predicted graph and choose the best one in evaluation
    perms: List[Tuple[Part]] = __generate_part_list_permutations(predicted_graph.get_parts())

    # Determine one part order for the target graph
    target_parts_order = perms[0]
    target_adj_matrix = target_graph.get_adjacency_matrix(target_parts_order)

    for perm in perms:
        predicted_adj_matrix = predicted_graph.get_adjacency_matrix(perm)
        score = np.sum(predicted_adj_matrix == target_adj_matrix)
        best_score = max(best_score, score)

    return best_score


def __generate_part_list_permutations(parts: Set[Part]) -> List[Tuple[Part]]:
    """
    Different instances of the same part type may be interchanged in the graph. This method computes all permutations
    of parts while taking this into account. This reduced the number of permutations.
    :param parts: Set of parts to compute permutations
    :return: List of part permutations
    """
    # split parts into sets of same part type
    equal_parts_sets: Dict[Part, Set[Part]] = {}
    for part in parts:
        for seen_part in equal_parts_sets.keys():
            if part.equivalent(seen_part):
                equal_parts_sets[seen_part].add(part)
                break
        else:
            equal_parts_sets[part] = {part}

    multi_occurrence_parts: List[Set[Part]] = [pset for pset in equal_parts_sets.values() if len(pset) > 1]
    single_occurrence_parts: List[Part] = [next(iter(pset)) for pset in equal_parts_sets.values() if len(pset) == 1]

    full_perms: List[Tuple[Part]] = [()]
    for mo_parts in multi_occurrence_parts:
        perms = list(permutations(mo_parts))
        full_perms = list(perms) if full_perms == [()] else [t1 + t2 for t1 in full_perms for t2 in perms]

    # Add single occurrence parts
    full_perms = [fp + tuple(single_occurrence_parts) for fp in full_perms]
    assert all([len(perm) == len(parts) for perm in full_perms]), 'Mismatching number of elements in permutation(s).'
    return full_perms

#print(evaluate(list(zip(X_val, y_val))))